In [ ]:
!nvidia-smi

In [ ]:
!conda info --envs

In [ ]:
import torch
from blora_utils import BLOCKS, filter_lora, scale_lora
import matplotlib.pyplot as plt
import cv2

from diffusers import (
    ControlNetModel, 
    DDIMScheduler,
    AutoencoderKL, 
    AutoPipelineForText2Image, 
    AutoPipelineForImage2Image,
    EulerAncestralDiscreteScheduler, 
    TCDScheduler
)
from diffusers.utils import load_image, make_image_grid

from transformers import CLIPVisionModelWithProjection

import numpy as np

import matplotlib.pyplot as plt

from PIL import Image

import torch


### Load Image

In [4]:
def apply_mask(image, mask):
    assert(image.shape[0:2]==mask.shape[0:2])
    # canny image
    alpha = mask / 255.0
    # Apply the mask
    image_on_mask = (alpha * image).astype(np.uint8)    
    return image_on_mask

def canny_edge_detection(img, low_threshold=50, high_threshold=200, apertureSize=3, L2gradient=True):
    # Apply Canny edge detection
    image_canny = cv2.Canny(img, low_threshold, high_threshold, apertureSize=apertureSize, L2gradient=L2gradient)
    # Convert the single-channel image to a three-channel image by stacking
    image_canny = np.stack((image_canny,)*3, axis=-1)
    return image_canny

### Handling of the generation the content

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("device:", device)

model_id_or_path = "SG161222/RealVisXL_V5.0_Lightning" #"stabilityai/stable-diffusion-xl-base-1.0" #
vae_id_or_path = "madebyollin/sdxl-vae-fp16-fix"

controlnet_canny = ControlNetModel.from_pretrained("xinsir/controlnet-canny-sdxl-1.0", weight_name="diffusion_pytorch_model_V2.safetensors", torch_dtype=torch.float16)

vae = AutoencoderKL.from_pretrained(vae_id_or_path, torch_dtype=torch.float16)

image_encoder = CLIPVisionModelWithProjection.from_pretrained(
    "h94/IP-Adapter",
    subfolder="models/image_encoder",
    torch_dtype=torch.float16,
)

pipe_tex2img = AutoPipelineForText2Image.from_pretrained(model_id_or_path,
                                                         controlnet=controlnet_canny,
                                                         vae=vae,
                                                         use_safetensors=True,
                                                         torch_dtype=torch.float16,
                                                         image_encoder=image_encoder,
                                                       )              

# Setting up of others parameters
pipe_tex2img.enable_xformers_memory_efficient_attention()
pipe_tex2img = pipe_tex2img.to(device)

# Update of the scheduler
pipe_tex2img.scheduler = TCDScheduler.from_config(pipe_tex2img.scheduler.config)

# Add IP-Adapter to the model
pipe_tex2img.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter-plus_sdxl_vit-h.safetensors")

In [ ]:
# loading and repositionong of image and mask #src/mea_genai/img/
image_input = load_image("img/image_6.png")
mask_image = load_image("img/image_6_mask.png") 

image_input_array = np.array(image_input)
mask_image_array = np.array(mask_image)

# Apply the mask
input_image_on_mask = apply_mask(image_input_array, mask_image_array)
image_canny = Image.fromarray(canny_edge_detection(input_image_on_mask, low_threshold=25, high_threshold=65, apertureSize=3, L2gradient=False))

# Load your images using load_image
images = [
    image_input,
    mask_image,
    image_canny,
]
make_image_grid(images, rows=1, cols=len(images), resize=512)

In [ ]:
# Image with nothing
prompt_per_ip_adapter_index = [(12, "The image shows a living room with a chair and a potted plant on the left side, a table with cups and saucers in the middle, and a carpet on the floor. On the wall behind the chair, there are three framed prints of a mountain scene, adding a touch of nature to the room"),
                               (13, "The image shows a living room with a grey couch sitting on top of a wooden floor, a carpet, a table with a lamp and other objects, a pot with a plant, a wall with a photo frame, and a glass window with trees visible outside."),
                               (14, "The image shows a brown leather accent chair in a living room with a white rug on the floor, a cupboard with books and a photo frame on top, a window with a curtain, and a wall in the background.")]
seed_per_ip_adapter_index = [(12, 33588),
 (14, 127871),
 (13, 366081),
 (12, 416179),
 (14, 466573),
 (13, 910052)]

# Number of images per batch
N = 2

# Loop over both lists
for index, (ip_adapter_index, prompt) in enumerate(prompt_per_ip_adapter_index):
    # Find the corresponding seed for this ip_adapter_index
    seeds_for_current_adapter = [seed for (seed_ip_adapter_index, seed) in seed_per_ip_adapter_index if seed_ip_adapter_index == ip_adapter_index]

    ip_adapter_image = load_image(f"img/inspiration/inspiration_{ip_adapter_index}.png") 
    
    # Iterate through the seeds for the current ip_adapter_index
    for seed in seeds_for_current_adapter:
        # Set the seed
        generator = torch.Generator(device='cuda').manual_seed(seed)
        print(f"seed: {seed}")

        # Set ip-adapter scaling
        pipe_tex2img.set_ip_adapter_scale(0.8)

        # Set control net scaling
        controlnet_conditioning_scale = 0.6

        # Print the prompt for reference
        print("prompt: ", prompt)

        # Run the inference
        images_output = pipe_tex2img(
            prompt=prompt,
            image=image_canny,
            ip_adapter_image=ip_adapter_image,
            controlnet_conditioning_scale=controlnet_conditioning_scale,
            num_inference_steps=10,
            num_images_per_prompt=N,
            guidance_scale=1.7,
            generator=generator
        ).images

        # Make image grid and save images
        make_image_grid(images_output, rows=1, cols=N, resize=512)
        
        # Save each image with unique filename
        for i, image_output in enumerate(images_output):
            plt.imsave(f'ai_image_sdxl_seed_{seed}_{index}_ip_adapter_{ip_adapter_index}_{i}.png', np.array(image_output).astype(np.uint8))

In [ ]:
# Loop over both lists
for index, (ip_adapter_index, prompt) in enumerate(prompt_per_ip_adapter_index):
    # Find the corresponding seed for this ip_adapter_index
    seeds_for_current_adapter = [seed for (seed_ip_adapter_index, seed) in seed_per_ip_adapter_index if seed_ip_adapter_index == ip_adapter_index]

    ip_adapter_image = load_image(f"img/inspiration/inspiration_{ip_adapter_index}.png") 
    
    # Iterate through the seeds for the current ip_adapter_index
    for seed in seeds_for_current_adapter:
        # Set the seed
        generator = torch.Generator(device='cuda').manual_seed(seed)
        print(f"seed: {seed}")

        # Set ip-adapter scaling
        pipe_tex2img.set_ip_adapter_scale(0)

        # Set control net scaling
        controlnet_conditioning_scale = 0.6

        # Print the prompt for reference
        print("prompt: ", prompt)

        # Run the inference
        images_output = pipe_tex2img(
            prompt=prompt,
            image=image_canny,
            ip_adapter_image=ip_adapter_image,
            controlnet_conditioning_scale=controlnet_conditioning_scale,
            num_inference_steps=10,
            num_images_per_prompt=N,
            guidance_scale=1.7,
            generator=generator
        ).images

        # Make image grid and save images
        make_image_grid(images_output, rows=1, cols=N, resize=512)
        
        # Save each image with unique filename
        for i, image_output in enumerate(images_output):
            plt.imsave(f'ai_image_sdxl_seed_{seed}_{index}_vanilla_{i}.png', np.array(image_output).astype(np.uint8))